# テーブルスキーマを検索して、BQ からデータを取得するエージェントの例

## 事前準備

In [1]:
import copy, datetime, json, os, pprint, time, uuid
import vertexai
from google.genai.types import Part, Content
from google.adk.agents.llm_agent import LlmAgent
from google.adk.artifacts import InMemoryArtifactService
from google.adk.memory.in_memory_memory_service import InMemoryMemoryService
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google import genai
from google.genai.types import (
    HttpOptions, GenerateContentConfig, Part, Content
)
from google.cloud import bigquery

[PROJECT_ID] = !gcloud config list --format 'value(core.project)'
LOCATION = 'us-central1'
vertexai.init(
    project=PROJECT_ID, location=LOCATION,
    staging_bucket=f'gs://{PROJECT_ID}'
)
os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID
os.environ['GOOGLE_CLOUD_LOCATION'] = LOCATION
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'True'

In [2]:
def generate_response(system_instruction, contents,
                      model='gemini-2.5-flash'):
    client = genai.Client(vertexai=True,
                          project=PROJECT_ID, location=LOCATION,
                          http_options=HttpOptions(api_version='v1'))
    response = client.models.generate_content(
        model=model,
        contents=contents,
        config=GenerateContentConfig(
            system_instruction=system_instruction,
            temperature=0.1,
            top_p=0.5,
            #response_mime_type='application/json',
            #response_schema=response_schema,
        )
    )
    return '\n'.join(
        [p.text for p in response.candidates[0].content.parts if p.text]
    )

In [3]:
class LocalApp:
    def __init__(self, agent, app_name='default_all', user_id='default_user'):
        self._agent = agent
        self._app_name = app_name
        self._user_id = user_id
        self._runner = Runner(
            app_name=self._app_name,
            agent=self._agent,
            artifact_service=InMemoryArtifactService(),
            session_service=InMemorySessionService(),
            memory_service=InMemoryMemoryService(),
        )
        self._session = None
        
    async def stream(self, query):
        if not self._session:
            self._session = await self._runner.session_service.create_session(
                app_name=self._app_name,
                user_id=self._user_id,
                session_id=uuid.uuid4().hex,
            )
        content = Content(role='user', parts=[Part.from_text(text=query)])
        async_events = self._runner.run_async(
            user_id=self._user_id,
            session_id=self._session.id,
            new_message=content,
        )
        result = []
        async for event in async_events:
            if DEBUG:
                print(f'===\n{event}\n===')
            if (event.content and event.content.parts):
                response = '\n'.join([p.text for p in event.content.parts if p.text])
                if response:
                    print(response)
                    result.append(response)
        return result

## スキーマデータの準備

### スキーマデータの例

本来は Cloud Datastore などの NoSQL DB に保存されている想定ですが、ここでは簡単のために変数 `scheme_data` に辞書形式で保存します。

In [4]:
schema_data = {
    'bigquery-public-data.new_york_taxi_trips.taxi_zone_geom':
'''
- zone_id : Unique ID number of each taxi zone. Corresponds with the pickup_location_id and dropoff_location_id in each of the trips tables
- zone_name : Full text name of the taxi zone
''',
    'bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022':
'''
- pickup_datetime : The date and time when the meter was engaged
- dropoff_datetime : The date and time when the meter was disengaged
- passenger_count : The number of passengers in the vehicle. This is a driver-entered value.
- trip_distance : The elapsed trip distance in miles reported by the taximeter.
- fare_amount : The time-and-distance fare calculated by the meter
- tip_amount : Tip amount. This field is automatically populated for credit card tips. Cash tips are not included.
- tolls_amount : Total amount of all tolls paid in trip.
- total_amount : The total amount charged to passengers. Does not include cash tips.
- pickup_location_id : TLC Taxi Zone in which the taximeter was engaged
- dropoff_location_id : TLC Taxi Zone in which the taximeter was disengaged
'''
}

スキーマデータを元に各テーブルの1行サマリーを生成しておきます。これを Agent のプロンプトに与えて、情報を取得するテーブルを選択する参考にさせます。

In [5]:
system_instruction = '''
Given table name and table schema, generate a one-sentence sammary of the table.
'''

table_info = ''
for table_name, table_schema in schema_data.items():
    contents = f'''
[table name]
{table_name}

[table schema]
{table_schema}
'''
    summary = generate_response(system_instruction, contents)
    table_info += f'{table_name}: {summary}\n'
    
print(table_info)

bigquery-public-data.new_york_taxi_trips.taxi_zone_geom: This table provides a mapping between unique IDs and the full text names of New York City taxi zones, which are referenced in taxi trip data.
bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022: This table contains detailed records of New York City yellow taxi trips from 2022, including pickup and dropoff times and locations, trip distance, passenger count, and various fare components like fare, tip, tolls, and total amount.



### ツール関数の定義

NoSQL DB（ここでは先に用意した変数）からテーブルスキーマを取得して返却するツールを定義します。

In [6]:
def table_schema_tool(table_name:str) -> str:
    """
    Return a table schema describing the data stored in table columns.

    Args:
        table_name: Table name

    Returns:
        Table schema string
    """    
    if table_name in schema_data.keys():
        return schema_data[table_name]
    else:
        return ''

SQL を用いて BQ からデータを取得するツールを定義します。

In [7]:
def bq_tool(query:str) -> str:
    """
    Get factual information from BigQuery using SQL queries.

    Args:
        query: SQL query in a single line for BigQuery.

    Returns:
        query result
    """
    
    if DEBUG:
        print(f'=== query ===\n{query}\n=============')
    client = bigquery.Client()
    query_job = client.query(query)
    result = query_job.result()
    result = [dict(row) for row in result]
    result = [{key: str(value) for key, value in raw.items()} for raw in result]
    return json.dumps(result)

## エージェント定義

In [8]:
instruction = f'''
You are a data analytics expert. Work on the following tasks.
    
[task]
Answer the question with the reason based on the data you get from BigQuery.

[condition]
- Base on the information in [table information], select BQ tables to get factual data.
- Use table_schema_tool to get table schema including column definitions to construct SQL query.
- Use bq_tool to get factual data from BigQuery using a SQL query.
- The answer and the reason must be based on the quantitative information in tables.
- Use concrete area names in the answer instead of zone_id or location_id.

[format instruction]
In Japanese. In plain text, no markdowns.
The output format is:
[回答]
<Your answer>

[理由]
<reason>


[table information]
{table_info}
'''

data_analytics_agent = LlmAgent(
    name='data_analytics_agent',
    model='gemini-2.5-flash',
    description='Agent to answer questions using BigQuery.',
    instruction=instruction,
    tools=[table_schema_tool, bq_tool],
)

### 実行例

`DEBUG = True` を指定して、処理中のイベントと使用する SQL query を表示します。

In [9]:
client = LocalApp(data_analytics_agent, 'Search Agent App', 'user00')

DEBUG = True
query = '''
チップがたくさんもらえる乗車場所を調べて。
'''
_ = await client.stream(query)

===
content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=FunctionCall(id='adk-4018b4ea-e4ac-49d4-bd55-86edbeaaa98d', args={'table_name': 'bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022'}, name='table_schema_tool'), function_response=None, text=None)], role='model') grounding_metadata=None partial=None turn_complete=None error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=GenerateContentResponseUsageMetadata(cache_tokens_details=None, cached_content_token_count=None, candidates_token_count=34, candidates_tokens_details=[ModalityTokenCount(modality=<MediaModality.TEXT: 'TEXT'>, token_count=34)], prompt_token_count=448, prompt_tokens_details=[ModalityTokenCount(modality=<MediaModality.TEXT: 'TEXT'>, token_count=448)], thoughts_token_count=277, tool_use_prompt_token_count=None, tool_use_prompt_tokens_details=

===
content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=FunctionCall(id='adk-66a2a60c-2277-483a-8077-cef8a353a27c', args={'table_name': 'bigquery-public-data.new_york_taxi_trips.taxi_zone_geom'}, name='table_schema_tool'), function_response=None, text=None)], role='model') grounding_metadata=None partial=None turn_complete=None error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=GenerateContentResponseUsageMetadata(cache_tokens_details=None, cached_content_token_count=None, candidates_token_count=28, candidates_tokens_details=[ModalityTokenCount(modality=<MediaModality.TEXT: 'TEXT'>, token_count=28)], prompt_token_count=675, prompt_tokens_details=[ModalityTokenCount(modality=<MediaModality.TEXT: 'TEXT'>, token_count=675)], thoughts_token_count=105, tool_use_prompt_token_count=None, tool_use_prompt_tokens_details=None, t

===
content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=FunctionCall(id='adk-91dccde9-e7b9-4d1f-b9ac-c7a0ca253ac2', args={'query': 'SELECT tzg.zone_name, AVG(t.tip_amount) AS avg_tip_amount FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022` AS t JOIN `bigquery-public-data.new_york_taxi_trips.taxi_zone_geom` AS tzg ON t.pickup_location_id = tzg.zone_id GROUP BY tzg.zone_name ORDER BY avg_tip_amount DESC LIMIT 5'}, name='bq_tool'), function_response=None, text=None)], role='model') grounding_metadata=None partial=None turn_complete=None error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=GenerateContentResponseUsageMetadata(cache_tokens_details=None, cached_content_token_count=None, candidates_token_count=117, candidates_tokens_details=[ModalityTokenCount(modality=<MediaModality.TEXT: 'TEXT'>, token_cou

In [10]:
query = '''
乗客数とチップの平均額を表にして、乗客数とチップの額に関連性があるか調べて。
'''
_ = await client.stream(query)

===
content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=FunctionCall(id='adk-6fd5625a-6430-4129-b160-39d9587e2394', args={'query': 'SELECT passenger_count, AVG(tip_amount) AS avg_tip_amount FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022` WHERE passenger_count IS NOT NULL AND passenger_count > 0 GROUP BY passenger_count ORDER BY passenger_count'}, name='bq_tool'), function_response=None, text=None)], role='model') grounding_metadata=None partial=None turn_complete=None error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=GenerateContentResponseUsageMetadata(cache_tokens_details=None, cached_content_token_count=None, candidates_token_count=74, candidates_tokens_details=[ModalityTokenCount(modality=<MediaModality.TEXT: 'TEXT'>, token_count=74)], prompt_token_count=1137, prompt_tokens_details=[ModalityT

In [12]:
query = '''
平均乗客数が多い乗車地域と時間帯の組み合わせについてトップ１０を教えて。
'''
_ = await client.stream(query)

===
content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=FunctionCall(id='adk-7e4c47cf-b26d-4104-b0e3-4718eb6f064a', args={'query': 'SELECT tzg.zone_name, EXTRACT(HOUR FROM t.pickup_datetime) AS pickup_hour, AVG(t.passenger_count) AS avg_passenger_count FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022` AS t JOIN `bigquery-public-data.new_york_taxi_trips.taxi_zone_geom` AS tzg ON t.pickup_location_id = tzg.zone_id GROUP BY tzg.zone_name, pickup_hour ORDER BY avg_passenger_count DESC LIMIT 10'}, name='bq_tool'), function_response=None, text=None)], role='model') grounding_metadata=None partial=None turn_complete=None error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=GenerateContentResponseUsageMetadata(cache_tokens_details=None, cached_content_token_count=None, candidates_token_count=137, candidates_t